In [66]:
import numpy as np
import matplotlib.gridspec
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed, IntSlider, HBox, Layout, Output, VBox
import ipywidgets as widgets
from scipy import signal

%matplotlib widget

In [67]:
def draw_circle(r):
    x1 = np.linspace(-r, r, 1000)
    y1 = np.sqrt(r**2-x1**2)
    x2 = np.linspace(r-r/1000, -r, 1000)
    y2 = np.sqrt(r**2-x2**2)
    x = np.concatenate([x1, x2])
    y = np.concatenate([y1, -y2])
    labels = ['-2j', '-1.5j', '-j', '-0.5j', '0', '0.5j', 'j','1.5j','2j']
    position=[-2,-1.5,-1,-0.5,0,0.5,1,1.5,2]
    plt.yticks(position, labels, rotation='horizontal')
    return x, y

In [68]:
plt.figure(figsize=(5,5))
uc = draw_circle(1)

x=[0.25,2/3]
y=[0,0]
# Pad margins so that markers don't get clipped by the axes
#plt.margins(5)
# Tweak spacing to prevent clipping of tick-labels
#plt.subplots_adjust(bottom=0.5)
plt.plot(uc[0], uc[1])
plt.plot(x,y)
plt.grid()
#plt.close('all')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [69]:
plt.close('all')
fig, ax = plt.subplots(figsize=(5,5))
uc = draw_circle(1)
fc = draw_circle(0.25)
sc = draw_circle(2/3)
inf_circ = draw_circle(5)
ax.plot(uc[0],uc[1], color='black', linewidth=0.5)
ax.plot(fc[0],fc[1],':')
ax.plot(sc[0], sc[1],':')
# Display the real and imaginary axes
ax.set_yticks([1e-4], minor=True)
ax.yaxis.grid(True, which='minor')
ax.set_xticks([1e-4], minor=True)
ax.xaxis.grid(True, which='minor')
ax.plot(x, y, 'xr')
#x = [0.5, 0.5, 0.5, 0.5, 0.5]
#y = -np.array([0.5, 0.5, 0.5, 0.5, 0.5])
#ax.plot(x, y, 'ob', fillstyle='none')
ax.set_xlim([-2,2])
ax.set_ylim([-2,2])
ax.fill(inf_circ[0],inf_circ[1],facecolor='lightgreen')
ax.fill(sc[0],sc[1],facecolor='lightskyblue')
ax.fill(fc[0],fc[1],facecolor='lightsalmon')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
def H(z):
    return (2/5)/(1-2/(3*z)-(3/20)/(1-1/(4*z)))

In [40]:
def step_f(causal,n):
    if causal:
        if n<0: return 0.
        else: return 1.
    else:
        if n<0: return 1.
        else: return 0.
        
#causal=True

In [48]:
def impulse_resp(region,n):
    if region==1:
        return (-2/5*(2/3)**n+3/20*(1/4)**n)*step_f(False,n)
    elif region==2:
        return (-4/3*(2/3)**n*step_f(False,n)-3/20*(1/4)**n*step_f(True,n))
    else:
        return (2/5*(2/3)**n-3/20*(1/4)**n)*step_f(True,n)

In [50]:
impulse_resp(1,-1)

-1.1102230246251565e-16

In [51]:
H(1)

2.999999999999999

In [52]:
H(1j)

(0.27921801954951125-0.22821929451763703j)

In [60]:
n=np.zeros(10)
for i in range(len(n)):
    n[i]=-4+i

In [61]:
n

array([-4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.,  5.])

In [62]:
r=np.zeros(10)
for i in range(len(n)):
    r[i]=impulse_resp(1,n[i])
    

In [63]:
r

array([ 3.63750000e+01,  8.25000000e+00,  1.50000000e+00, -1.11022302e-16,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00])

In [76]:
plt.close('all')
fig, ax = plt.subplots(figsize=(5,5))
uc = draw_circle(1)
fc = draw_circle(0.25)
sc = draw_circle(2/3)
inf_circ = draw_circle(5)
ax.plot(uc[0],uc[1], color='black', linewidth=0.5)
ax.plot(fc[0],fc[1],':')
ax.plot(sc[0], sc[1],':')
# Display the real and imaginary axes
ax.set_yticks([1e-4], minor=True)
ax.yaxis.grid(True, which='minor')
ax.set_xticks([1e-4], minor=True)
ax.xaxis.grid(True, which='minor')
ax.plot(x, y, 'xr')
#x = [0.5, 0.5, 0.5, 0.5, 0.5]
#y = -np.array([0.5, 0.5, 0.5, 0.5, 0.5])
#ax.plot(x, y, 'ob', fillstyle='none')
ax.set_xlim([-2,2])
ax.set_ylim([-2,2])
ax.fill(inf_circ[0],inf_circ[1],facecolor='lightgreen')
ax.fill(sc[0],sc[1],facecolor='lightskyblue')
ax.fill(fc[0],fc[1],facecolor='lightsalmon')
fig.plot(1,1)
#fig.plot(n,r,color='blue', linewidth=0.4)
plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AttributeError: 'Figure' object has no attribute 'plot'

In [107]:
class Region_of_Convergence():
    def __init__(self,causal=True,region=1,integer=11):
        
        self.out = Output(layout={'width': '980px', 'height': '450px'})
        self.axs=[]
        
        #Defining the boolean casual and the region(1:inside, 2:between and 3:outside)
        self.causal=causal
        self.region=region
        
        #Defining the integers, always has to be an odd number
        n=np.zeros(integer)
        for i in range(integer):
            n[i]=(integer-1)/2+i
            
        #Defining the array to store the impulse response
        h=np.zeros(integer)
        
        #Defining the poles
        p_x=[0.25,2/3]
        p_y=[0,0]
        
        
        # Debug text field
        self.tx_deb = self.axs[0].text(-2,0,'')
        
        #Inizializate the figure
        self.init_figure(p_x, p_y)
        
    def init_figure(self, p_x, p_y):
        with self.out:
            
            # Create the Complex plane plot and small plots for H and h
            self.fig = plt.figure(figsize=(8, 4))
            self.gs = self.fig.add_gridspec(2, 2)
            
            self.axs[0]=self.fig.add_subplot(self.gs[:,0])
            self.axs[0].set_title("Regions of Convergence")
            
            self.axs[1]=self.fig.add_subplot(self.gs[0:, -1])
            self.axs[1].set_title("Z transform")
            
            self.axs[2]=self.fig.add_subplot(self.gs[1:, -1])
            self.axs[2].set_title("Impulse response")
            
            #Defining the unit circle and regions
            uc = self.draw_circle(1)
            fc = self.draw_circle(0.25)
            sc = self.draw_circle(2/3)
            inf_circ = self.draw_circle(5)
            
            # Draw the unit circle and regions filled
            self.axs[0].plot(uc[0], uc[1], color='black', linewidth='0.5')
            self.axs[0].plot(fc[0],fc[1],':')
            self.axs[0].plot(sc[0], sc[1],':')
            self.axs[0].fill(inf_circ[0],inf_circ[1],facecolor='lightgreen')
            self.axs[0].fill(sc[0],sc[1],facecolor='lightskyblue')
            self.axs[0].fill(fc[0],fc[1],facecolor='lightsalmon')
            #if not self.discrete_mode:
                #self.axs[0].lines[0].set_visible(False)
                
            
            # Add zeros and poles
            #self.axs[0].plot(z_x, z_y, 'xr', label='Zeros')
            self.axs[0].plot(p_x, p_y, 'ob', fillstyle='none', label='Poles')
            self.axs[0].set_xlim([-2, 2])
            self.axs[0].set_ylim([-2, 2])
            
            # Display the real and imaginary axes
            self.axs[0].set_yticks([1e-4], minor=True)
            self.axs[0].yaxis.grid(True, which='minor')
            self.axs[0].set_xticks([1e-4], minor=True)
            self.axs[0].xaxis.grid(True, which='minor')
            self.axs[0].set_title('Regions of convergence')
            self.axs[0].set_xlabel('Re')
            self.axs[0].set_ylabel('Im')
            
            # Enable the legend
            self.axs[0].legend()  
    
    def H(z):
        return (2/5)/(1-2/(3*z)-(3/20)/(1-1/(4*z)))
        
    def step_f(self):
        if self.causal:
            if n<0: return 0.
            else: return 1.
        else:
            if n<0: return 1.
            else: return 0.       
    
    def impulse_resp(self):
        if self.region==1:
            return (-2/5*(2/3)**n+3/20*(1/4)**n)*self.step_f(False,n)
        elif self.region==2:
            return (-4/3*(2/3)**n*self.step_f(False,n)-3/20*(1/4)**n*self.step_f(True,n))
        else:
            return (2/5*(2/3)**n-3/20*(1/4)**n)*self.step_f(True,n)
    
    def draw_circle(r):
        x1 = np.linspace(-r, r, 1000)
        y1 = np.sqrt(r**2-x1**2)
        x2 = np.linspace(r-r/1000, -r, 1000)
        y2 = np.sqrt(r**2-x2**2)
        x = np.concatenate([x1, x2])
        y = np.concatenate([y1, -y2])
        labels = ['-2j', '-1.5j', '-j', '-0.5j', '0', '0.5j', 'j','1.5j','2j']
        position=[-2,-1.5,-1,-0.5,0,0.5,1,1.5,2]
        plt.yticks(position, labels, rotation='horizontal')
        return x, y

In [108]:
plt.close('all')
roc=Region_of_Convergence()


IndexError: list index out of range

In [85]:
roc.axs[1]

IndexError: list index out of range

In [103]:
fig3 = plt.figure(figsize=(10, 5),constrained_layout=True)
gs = fig3.add_gridspec(2, 2)
f3_ax1 = fig3.add_subplot(gs[:, 0])
f3_ax1.set_title('gs[0, :]')
f3_ax2 = fig3.add_subplot(gs[0:, -1])
f3_ax2.set_title('gs[1, :-1]')
f3_ax3 = fig3.add_subplot(gs[1:, -1])
f3_ax3.set_title('gs[1:, -1]')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'gs[1:, -1]')